# Quantum Classfier

In [ ]:
!pip install qiskit qiskit_machine_learning
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.neural_networks import EstimatorQNN
import numpy as np
# from qiskit.primitives import StatevectorEstimator
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_machine_learning.gradients import ParamShiftEstimatorGradient

In [ ]:

class HybridModel:
    def __init__(self, ansatz_circuit, num_qubits):

        # we inicialize the variables
        self.num_qubits = num_qubits

        '''now we perform data encoding using ZZFetureMap, briefly it consist of three layers
        1. hadamard - puts all the qubits in superposition state |+>
        2. Z-Rotations - rotates each qubit individually based on input features Rz(x)
        3. ZZ-Rotations - it entangles qubits by rotating them based on the relationship between two features

        reps=1, we do the circuit of encoding only once. We can try and change that to 2 but it gets deeper
        '''
        self.feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1)

        # Initalizing quantum circuit. Here we are connecting our feature map (data) and ansatz
        self.qc = QuantumCircuit(num_qubits)
        self.qc.compose(self.feature_map,qubits=range(num_qubits), inplace=True)
        self.qc.compose(ansatz_circuit, inplace=True)


        '''
        So that is a crucial step. Firstly, we inicialize parameters. Our quantum model cannot tell whether the number came from ansatz or feature.
        That is why here we sort them into two lists. If the number came from feature_map, then it will be a feature and the other way around.

        '''
        final_circuit_params = self.qc.parameters

        # creating dicts with names of the variables to then check them in our loop
        feature_map_names = {p.name for p in self.feature_map.parameters}
        ansatz_names = {p.name for p in ansatz_circuit.parameters}

        self.final_input_params = []
        self.final_weight_params = []

        for p in final_circuit_params:
            if p.name in feature_map_names:
                self.final_input_params.append(p)
            elif p.name in ansatz_names:
                self.final_weight_params.append(p)

        observable = SparsePauliOp.from_list([("I" * (num_qubits - 1) + "Z", 1)])

        '''Estimator takes ansatz, observables and parameters (data and weights), returns the Expectation value.'''


        # estimator = StatevectorEstimator()
        estimator = Estimator()
        # creating a gradient for backward
        gradient = ParamShiftEstimatorGradient(estimator)
        self.qnn = EstimatorQNN(
            circuit=self.qc,
            observables=observable,
            input_params=self.final_input_params,
            weight_params=self.final_weight_params,
            estimator=estimator,
            gradient = gradient
        )


    # forward: gives the Estimator data and weights, sets the qubit angles, runs gates and measures the result
    def forward(self, x, weights):
        return self.qnn.forward(x, weights)

    # backward In Quantum ML it actually runs forward once again but with a little change
    # Firstly it shifts the angles by pi/2 and runs the circuit, then -pi/2 and runs the circuit
    # It calculates the difference and it's the gradient, we only return weight_grads because we dont need input_grads
    def backward(self, x, weights):
        _, weight_grads = self.qnn.backward(x, weights)
        if weight_grads is None:
            # If it fails, return zeros to prevent the loop from crashing
            print("Warning: Gradients were None. Returning Zeros.")
            return np.zeros((x.shape[0], len(weights)))
        return weight_grads


In [ ]:
my_ansatz = ansatz_A1(4, 4)
qnn = HybridModel(
    ansatz_circuit=my_ansatz,
    num_qubits=5,
)


# Inicializing the weights
num_weights = qnn.qnn.num_weights
rng = np.random.default_rng(seed=42)
weights = 2 * np.pi * rng.random(num_weights)
weights = weights.flatten()

print(f"Weights initialized. Shape: {weights.shape}")